# Singular Value Decomposition
*Curtis Miller*

**Singular value decomposition (SVD)** is a matrix decomposition technique from linear algebra. A matrix $X$ is expressed as the product of two unitary matrices $U$ and $V$ (with unitary meaning that $U U^T = V^T V = I$) and a matrix $S$ whose entries are zero except for the diagonal entries, which could be positive numbers from largest (in the top left corner) to smallest (possibly 0), so $X = USV^T$. The diagonal entries of $S$ are the **singular values** of $X$.

The SVD contains all information about a matrix and encodes it in a simple manner. It is an important decomposition technique. PCA, for example, depends on the SVD of the data matrix.

$U$ and $V$ are **dense** (meaning that most if not all of their entries are not zero) while $S$ is sparse. We would need to store every column of $U$ and column of $V$ that corresponds to a non-zero singular value. But we can store less information by making some of the singular values of $S$ zero, and keeping only some of the columns of $U$ and $V$. After doing this we can use the information we kept to approximate the original matrix $X$ with arbitrary precision. (The more singular values kept, the better the approximation.)

If $X$ is an $m \times n$ matrix, then $U$ is $m \times m$, $V$ is $n \times n$, and $S$ is $m \times n$. If $X$ has $r$ non-zero singular values, the **compact SVD** finds $X = \tilde{U} \tilde{S} \tilde{V}^T$, where $\tilde{U}$ is a $m \times r$ matrix, $\tilde{S}$ is a $r \times r$ matrix, and $\tilde{V}$ is a $n \times r$ matrix. The compact SVD will perfectly reconstruct $X$ and by reducing the number of non-zero singular values (that is, reduce $r$), we reduce the amount of information we need to store.

In this notebook I demonstrate how SVD can be used for image compression. We will be compressing this image:

In [ ]:
from PIL import Image
import numpy as np
import numpy.linalg as ln
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
frog = np.array(Image.open("frog.png").convert("RGB")) / 255    # The last division to force numbers to be in [0,1]
plt.imshow(frog)

In [ ]:
plt.imshow(frog[:, :, 0], cmap="Reds")

In [ ]:
plt.imshow(frog[:, :, 1], cmap="Greens")

In [ ]:
plt.imshow(frog[:, :, 2], cmap="Blues")

What if we wanted an SVD decomposition of the upper 5x7 corner of red values?

In [ ]:
plt.imshow(frog[:5, :7, 0], cmap='Reds')

In [ ]:
m, n = 5, 7
U, S, V = ln.svd(frog[:m, :n, 0])    # The full SVD
U

In [ ]:
S    # The singular values

In [ ]:
V.T    # ln.svd returns the transpose of V as described above

All singular values are non-zero. We can form a compact SVD of this matrix to represent it in a smaller space, though. We may, for example, use only the first two singular values.

In [ ]:
r = 2
nU, nS, nV = U[:m, :r], S[:r], V[:r, :n]
nU

In [ ]:
nS

In [ ]:
nV.T

In [ ]:
nX = nU.dot(np.diag(nS)).dot(nV)
nX

In [ ]:
nX.shape

In [ ]:
plt.imshow(nX, cmap="Reds")

The following function finds the matrices of the compact SVD of a matrix for approximation.

In [ ]:
def approx_compact_svd(a, r):
    """Finds U S V for an approximation of matrix a with rank r"""
    m, n = a.shape
    U, S, V = ln.svd(a)
    return (U[:m, :r], S[:r], V.T[:n, :r])

This function uses the above function for reconstructing an image based on approximations of this nature.

In [ ]:
def svd_compress_image(img, ranks):
    """SVD compression of img; ranks should be a tuple of three values for how many singular values for
       R, G, B channels, respectively"""
    channels = [img[:, :, 0], img[:, :, 1], img[:, :, 2]]
    new_channels = list()
    for a, r in zip(channels, ranks):
        U, S, V = approx_compact_svd(a, r)
        new_channels.append(U.dot(np.diag(S)).dot(V.T))
    return np.array(new_channels).transpose((1, 2, 0))

In [ ]:
newfrog = svd_compress_image(frog, (10, 10, 10))
plt.imshow(newfrog)

In [ ]:
newfrog2 = svd_compress_image(frog, (200, 200, 200))    # Will 200 values be better?
plt.imshow(newfrog2)

The compression quality depends on how many singular values are kept. Choosing this number wisely is important.